#### Saturday, February 14, 2026

This all runs in one pass, 

# 07 - Paper Trading System: From Backtest to Live Simulation

This capstone notebook ties together **everything from notebooks 01-06** into a unified
paper trading system. It replays historical data bar-by-bar (simulation mode) or connects
to the **Alpaca API** for real paper trading.

---

## What You'll Build

1. **Market Data Engine** -- Unified interface for simulation replay and live data
2. **Signal Integration Hub** -- All 10 signals from NB06 computed on streaming bars
3. **Regime-Aware Scoring** -- Adaptive composite scoring for live decisions
4. **Order Management System** -- Paper position tracking with Alpaca API integration
5. **Pre-Market Scanner** -- Morning stock ranking by signal strength
6. **Live Trading Loop** -- Event-driven bar -> signal -> trade pipeline
7. **Interactive Dashboard** -- Plotly 4-panel real-time visualization
8. **Trade Persistence** -- SQLite database for trade history
9. **QuantStats Tearsheet** -- Professional performance reporting vs SPY benchmark
10. **Statistical Validation** -- Bootstrap Sharpe CI, t-tests, distribution analysis
11. **Risk Management Dashboard** -- Drawdown zones, concentration, rolling metrics

---

## System Architecture

```
                    +-------------------------------------+
                    |         Pre-Market Scanner           |
                    |  (overnight news, Chronos forecasts) |
                    +-----------------+-------------------+
                                      | ranked universe
                    +-----------------v-------------------+
                    |        Market Data Engine            |
                    |  SimulationReplayer <--> Alpaca API  |
                    +-----------------+-------------------+
                                      | bar events
                    +-----------------v-------------------+
                    |        Signal Integration Hub       |
                    |  Technical | Momentum | Sentiment   |
                    |  Chronos   | Volume   | Mean-Rev    |
                    +-----------------+-------------------+
                                      | 10 normalized signals
                    +-----------------v-------------------+
                    |     Regime-Aware Composite Score     |
                    |  ADX regime -> adaptive weighting    |
                    +-----------------+-------------------+
                                      | score + agreement
                    +-----------------v-------------------+
                    |     Order Management System          |
                    |  entries, exits, stops, sizing        |
                    |  Internal ledger <--> Alpaca Orders  |
                    +-----------------+-------------------+
                                      | trades
              +----------+------------+-----------+
              v                       v                    v
     +---------------+    +---------------+    +---------------+
     |  SQLite DB    |    |  QuantStats   |    |  Risk Dash    |
     |  (peewee)     |    |  Tearsheet    |    |  (Plotly)     |
     +---------------+    +---------------+    +---------------+
```

---

## Design Philosophy

| Principle | Implementation |
|-----------|---------------|
| **Simulation-first** | Works without any API keys -- replays yfinance bar-by-bar |
| **Alpaca-ready** | Set `ALPACA_KEY`/`ALPACA_SECRET` env vars to paper trade live |
| **Self-contained** | All classes defined fresh (no cross-notebook imports) |
| **Lazy GPU loading** | Chronos/sentiment models load only when explicitly requested |
| **10 normalized signals** | Same -1 to +1 system from NB06, regime-adaptive weighting |

---
## 1. Configuration & Environment Setup

Install `alpaca-py` for paper trading API access (optional -- simulation mode
works without it). All other dependencies are already installed.

In [1]:
# Uncomment to install optional dependencies:
# !pip install alpaca-py peewee

import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
import torch
import time
import json as _json
import sqlite3
import warnings
import os
from dotenv import load_dotenv
load_dotenv()
from dataclasses import dataclass, field
from typing import Optional, Dict, List, Tuple
from collections import defaultdict
from datetime import datetime, timedelta

from ta.trend import EMAIndicator, SMAIndicator, MACD, ADXIndicator
from ta.momentum import RSIIndicator
from ta.volatility import BollingerBands, AverageTrueRange

# Optional imports with graceful fallback
HAS_PLOTLY = False
try:
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    HAS_PLOTLY = True
except ImportError:
    print("plotly not available -- install with: pip install plotly")

HAS_QUANTSTATS = False
try:
    import quantstats as qs
    HAS_QUANTSTATS = True
except ImportError:
    print("quantstats not available -- install with: pip install quantstats")

HAS_ALPACA = False
try:
    from alpaca.trading.client import TradingClient
    from alpaca.trading.requests import MarketOrderRequest
    from alpaca.trading.enums import OrderSide, TimeInForce, OrderStatus
    HAS_ALPACA = True
except ImportError:
    pass

warnings.filterwarnings("ignore", category=FutureWarning)
plt.style.use("dark_background")
sns.set_palette("bright")

print(f"PyTorch: {torch.__version__} | CUDA: {torch.cuda.is_available()}")
print(f"  alpaca-py: {'yes' if HAS_ALPACA else 'no (simulation only)'}")
print(f"  plotly:    {'yes' if HAS_PLOTLY else 'no'}")
print(f"  quantstats: {'yes' if HAS_QUANTSTATS else 'no'}")

PyTorch: 2.6.0+cu124 | CUDA: True
  alpaca-py: yes
  plotly:    yes
  quantstats: yes


In [2]:
@dataclass
class TradingConfig:
    """Central configuration for the paper trading system."""
    # Mode -- auto-detect: use live paper trading if Alpaca credentials are set
    simulation_mode: bool = field(
        default_factory=lambda: not bool(os.environ.get("ALPACA_KEY"))
    )

    # Universe
    universe: List[str] = field(default_factory=lambda: [
        "NVDA", "AAPL", "TSLA", "AMD", "MSFT", "META", "AMZN", "GOOGL"
    ])
    benchmark: str = "SPY"

    # Capital & Risk
    initial_capital: float = 100_000.0
    max_positions: int = 5
    risk_per_trade_pct: float = 2.0        # percent of capital risked per trade
    max_portfolio_risk: float = 0.06
    max_per_stock_pct: float = 20.0        # max allocation per stock (%)

    # Signal thresholds
    entry_threshold: float = 0.3
    exit_threshold: float = -0.1
    min_agreement: float = 0.5             # min signal agreement ratio for entry

    # Position management
    atr_stop_mult: float = 2.0             # ATR multiplier for stop loss
    atr_target_mult: float = 3.0           # ATR multiplier for take profit
    max_hold_bars: int = 20                # max bars before forced exit
    warmup_bars: int = 60                  # bars needed before signals are valid

    # Execution
    slippage_bps: float = 5.0
    commission_per_share: float = 0.005

    # Persistence
    db_path: str = "paper_trades.db"

    @property
    def slippage_pct(self) -> float:
        return self.slippage_bps / 10_000

    # Alpaca (from environment variables)
    alpaca_key: str = field(default_factory=lambda: os.environ.get("ALPACA_KEY", ""))
    alpaca_secret: str = field(default_factory=lambda: os.environ.get("ALPACA_SECRET", ""))
    alpaca_paper: bool = True

CONFIG = TradingConfig()
print(f"Trading Config: {'SIMULATION' if CONFIG.simulation_mode else 'LIVE PAPER'} mode")
print(f"  Universe: {CONFIG.universe}")
print(f"  Capital: ${CONFIG.initial_capital:,.0f} | Max positions: {CONFIG.max_positions}")

# Try to connect to Alpaca if credentials are available
alpaca_client = None
if not CONFIG.simulation_mode and HAS_ALPACA and CONFIG.alpaca_key:
    try:
        alpaca_client = TradingClient(CONFIG.alpaca_key, CONFIG.alpaca_secret, paper=True)
        account = alpaca_client.get_account()
        print(f"--- Alpaca Connected ---")
        print(f"  Buying power: ${float(account.buying_power):,.2f}")
        print(f"  Equity:       ${float(account.equity):,.2f}")
        print(f"  Day trades:   {account.daytrade_count}/3")
    except Exception as e:
        print(f"Alpaca connection failed: {e}")
        print("Falling back to simulation mode.")
        CONFIG.simulation_mode = True
else:
    if not CONFIG.simulation_mode:
        print("No Alpaca credentials. Set ALPACA_KEY and ALPACA_SECRET env vars.")
        CONFIG.simulation_mode = True
    print("  (Set simulation_mode=False and provide Alpaca keys for live paper trading)")

Trading Config: LIVE PAPER mode
  Universe: ['NVDA', 'AAPL', 'TSLA', 'AMD', 'MSFT', 'META', 'AMZN', 'GOOGL']
  Capital: $100,000 | Max positions: 5
--- Alpaca Connected ---
  Buying power: $200,000.00
  Equity:       $100,000.00
  Day trades:   0/3


---
## 2. Market Data Engine

The `SimulationReplayer` downloads historical data via yfinance and replays it
bar-by-bar, simulating a live feed. The `MarketDataEngine` wraps this with a
unified interface that works for both simulation and live modes.

In [3]:
@dataclass
class BarEvent:
    """A single price bar event."""
    ticker: str
    timestamp: pd.Timestamp
    open: float
    high: float
    low: float
    close: float
    volume: float
    bar_index: int


class SimulationReplayer:
    """Replays historical yfinance data bar-by-bar, simulating a live feed."""

    def __init__(self, tickers, period="1y", interval="1d"):
        self.tickers = tickers
        self.data = {}
        self.bar_index = 0

        print(f"Loading simulation data for {len(tickers)} tickers...")
        for ticker in tickers:
            df = yf.download(ticker, period=period, interval=interval, progress=False)
            if isinstance(df.columns, pd.MultiIndex):
                df.columns = df.columns.get_level_values(0)
            self.data[ticker] = df
            print(f"  {ticker}: {len(df)} bars | ${df['Close'].iloc[-1]:.2f}")

        common_dates = set.intersection(*[set(df.index) for df in self.data.values()])
        self.common_dates = sorted(common_dates)
        self.max_bars = len(self.common_dates)
        print(f"\n{self.max_bars} common trading days loaded.")

    def get_history(self, ticker, up_to_bar):
        """Get all data up to and including the specified bar index."""
        if ticker not in self.data:
            return None
        date = self.common_dates[up_to_bar]
        df = self.data[ticker]
        return df[df.index <= date].copy()

    def get_bar(self, ticker, bar_index):
        """Get a single bar event."""
        date = self.common_dates[bar_index]
        df = self.data[ticker]
        if date not in df.index:
            return None
        row = df.loc[date]
        return BarEvent(
            ticker=ticker, timestamp=date,
            open=float(row["Open"]), high=float(row["High"]),
            low=float(row["Low"]), close=float(row["Close"]),
            volume=float(row["Volume"]), bar_index=bar_index,
        )

    def iter_bars(self, warmup=60):
        """Iterate through bars, yielding dict of {ticker: BarEvent} per timestep."""
        for i in range(warmup, self.max_bars):
            bars = {}
            for ticker in self.tickers:
                bar = self.get_bar(ticker, i)
                if bar:
                    bars[ticker] = bar
            yield i, bars


class MarketDataEngine:
    """Unified interface for simulation and live data."""

    def __init__(self, config):
        self.config = config
        self.replayer = None
        self._live_cache = {}

        if config.simulation_mode:
            self.replayer = SimulationReplayer(
                config.universe + [config.benchmark],
                period="1y", interval="1d"
            )

    def get_history(self, ticker, up_to_bar=None):
        """Get historical data for a ticker."""
        if self.config.simulation_mode and self.replayer:
            if up_to_bar is not None:
                return self.replayer.get_history(ticker, up_to_bar)
            return self.replayer.data.get(ticker)
        # Live mode: fetch from yfinance (cached per session)
        if ticker not in self._live_cache:
            df = yf.download(ticker, period="1y", interval="1d", progress=False)
            if isinstance(df.columns, pd.MultiIndex):
                df.columns = df.columns.get_level_values(0)
            self._live_cache[ticker] = df if len(df) > 0 else None
        return self._live_cache[ticker]

    def iter_bars(self, warmup=60):
        """Iterate through bar events."""
        if self.config.simulation_mode and self.replayer:
            yield from self.replayer.iter_bars(warmup=warmup)


data_engine = MarketDataEngine(CONFIG)
print(f"\nMarketDataEngine ready ({'simulation' if CONFIG.simulation_mode else 'live'} mode)")


MarketDataEngine ready (live mode)


---
## 3. Signal Integration Hub

All 10 signals from Notebook 06, adapted for streaming bar-by-bar computation.
Each signal is normalized to **-1 (bearish)** to **+1 (bullish)**.

| Category | Signals | Source |
|----------|---------|--------|
| **Trend** | EMA alignment, MACD histogram | NB03/06 |
| **Momentum** | 5-day momentum, 20-day momentum, volume surge | NB06 |
| **Reversion** | RSI, Bollinger Band position, 1-day mean reversion | NB03/06 |
| **Sentiment** | FinBERT ensemble (or momentum proxy) | NB02 |
| **Forecast** | Chronos prediction (or trend proxy) | NB04 |

In [4]:
class TechnicalSignals:
    """Generate technical analysis signals (from Notebook 06).
    All signals normalized to -1 (bearish) to +1 (bullish)."""

    @staticmethod
    def compute(df):
        """Add all technical signals to a DataFrame."""
        out = df.copy()

        out["EMA_9"] = EMAIndicator(df["Close"], window=9).ema_indicator()
        out["EMA_21"] = EMAIndicator(df["Close"], window=21).ema_indicator()
        out["EMA_50"] = EMAIndicator(df["Close"], window=50).ema_indicator()
        out["RSI"] = RSIIndicator(df["Close"], window=14).rsi()

        macd = MACD(df["Close"])
        out["MACD_Hist"] = macd.macd_diff()

        bb = BollingerBands(df["Close"])
        out["BB_Upper"] = bb.bollinger_hband()
        out["BB_Lower"] = bb.bollinger_lband()
        out["BB_Middle"] = bb.bollinger_mavg()

        out["ATR"] = AverageTrueRange(df["High"], df["Low"], df["Close"]).average_true_range()
        out["ADX"] = ADXIndicator(df["High"], df["Low"], df["Close"]).adx()

        # --- Normalized Signals ---
        ema_fast_above_slow = (out["EMA_9"] > out["EMA_21"]).astype(float)
        ema_slow_above_long = (out["EMA_21"] > out["EMA_50"]).astype(float)
        out["sig_ema_trend"] = ema_fast_above_slow + ema_slow_above_long - 1.0

        out["sig_rsi"] = -1.0 * (out["RSI"] - 50) / 50
        out["sig_rsi"] = out["sig_rsi"].clip(-1, 1)

        macd_max = out["MACD_Hist"].rolling(50).apply(
            lambda x: max(abs(x.max()), abs(x.min()), 1e-8))
        out["sig_macd"] = (out["MACD_Hist"] / macd_max).clip(-1, 1)

        bb_range = out["BB_Upper"] - out["BB_Lower"]
        bb_position = (out["Close"] - out["BB_Lower"]) / bb_range.replace(0, np.nan)
        out["sig_bb"] = -1.0 * (bb_position * 2 - 1)
        out["sig_bb"] = out["sig_bb"].clip(-1, 1)

        out["trend_strength"] = (out["ADX"] / 50).clip(0, 1)
        return out.dropna()


class MomentumSignals:
    """Price momentum and mean-reversion signals."""

    @staticmethod
    def compute(df):
        out = df.copy()
        out["ret_1d"] = out["Close"].pct_change(1)
        out["ret_5d"] = out["Close"].pct_change(5)
        out["ret_20d"] = out["Close"].pct_change(20)

        ret_5d_std = out["ret_5d"].rolling(60).std()
        out["sig_momentum_5d"] = (out["ret_5d"] / ret_5d_std.replace(0, np.nan)).clip(-2, 2) / 2

        ret_20d_std = out["ret_20d"].rolling(60).std()
        out["sig_momentum_20d"] = (out["ret_20d"] / ret_20d_std.replace(0, np.nan)).clip(-2, 2) / 2

        ret_1d_std = out["ret_1d"].rolling(20).std()
        z_score = out["ret_1d"] / ret_1d_std.replace(0, np.nan)
        out["sig_mean_reversion"] = (-z_score).clip(-1, 1)

        vol_avg = out["Volume"].rolling(20).mean()
        vol_ratio = out["Volume"] / vol_avg.replace(0, np.nan)
        out["sig_volume"] = (np.sign(out["ret_1d"]) * (vol_ratio - 1) / 2).clip(-1, 1)
        return out.dropna()


class LiveSentimentSignal:
    """Live sentiment signal using FinBERT (from NB02).
    Loads model lazily. Falls back to momentum proxy if unavailable."""

    def __init__(self):
        self.pipeline = None
        self.loaded = False

    def load_model(self):
        """Explicitly load the FinBERT model onto GPU."""
        try:
            from transformers import pipeline as hf_pipeline
            self.pipeline = hf_pipeline(
                "sentiment-analysis", model="ProsusAI/finbert",
                device=0 if torch.cuda.is_available() else -1,
                truncation=True, max_length=512,
            )
            self.loaded = True
            print("FinBERT loaded for live sentiment.")
        except Exception as e:
            print(f"FinBERT load failed: {e}. Using momentum proxy.")
            self.loaded = False

    def compute(self, history_df, headlines=None):
        """Compute sentiment signal. Uses FinBERT if loaded, else momentum proxy."""
        if self.loaded and self.pipeline and headlines:
            scores = []
            for headline in headlines[:20]:
                result = self.pipeline(headline)[0]
                label = result["label"].lower()
                score = result["score"]
                if "positive" in label:
                    scores.append(score)
                elif "negative" in label:
                    scores.append(-score)
                else:
                    scores.append(0.0)
            return float(np.clip(np.mean(scores), -1, 1)) if scores else 0.0

        ret_5d = history_df["Close"].pct_change(5).iloc[-1]
        return float(np.clip(ret_5d * 5, -1, 1))


class LiveChronosSignal:
    """Live Chronos forecast signal (from NB04).
    Loads model lazily. Falls back to trend proxy if unavailable."""

    def __init__(self):
        self.model = None
        self.loaded = False

    def load_model(self, model_name="amazon/chronos-bolt-small"):
        """Explicitly load the Chronos model onto GPU."""
        try:
            from chronos import ChronosBoltPipeline
            self.model = ChronosBoltPipeline.from_pretrained(
                model_name,
                device_map="cuda" if torch.cuda.is_available() else "cpu",
                torch_dtype=torch.float32,
            )
            self.loaded = True
            print(f"Chronos loaded: {model_name}")
        except Exception as e:
            print(f"Chronos load failed: {e}. Using trend proxy.")
            self.loaded = False

    def compute(self, history_df, horizon=5):
        """Compute forecast signal. +1 if price predicted up, -1 for down."""
        if self.loaded and self.model:
            try:
                prices = history_df["Close"].values[-200:]
                context = torch.tensor(prices, dtype=torch.float32)
                forecast = self.model.predict(
                    context=context, prediction_length=horizon, num_samples=100)
                forecast_np = forecast.numpy()
                median_end = np.median(forecast_np[:, -1])
                current = prices[-1]
                pct_change = (median_end - current) / current
                return float(np.clip(pct_change * 20, -1, 1))
            except Exception:
                pass
        ret_20d = history_df["Close"].pct_change(20).iloc[-1]
        return float(np.clip(ret_20d * 3, -1, 1))


class SignalHub:
    """Aggregates all signal sources into a unified signal dictionary."""

    def __init__(self, config, use_sentiment=False, use_chronos=False):
        self.config = config
        self.sentiment_signal = LiveSentimentSignal()
        self.chronos_signal = LiveChronosSignal()
        if use_sentiment:
            self.sentiment_signal.load_model()
        if use_chronos:
            self.chronos_signal.load_model()

    def compute(self, history_df, headlines=None):
        """Compute all signals from price history. Returns dict for latest bar."""
        tech = TechnicalSignals.compute(history_df)
        mom = MomentumSignals.compute(history_df)
        if len(tech) == 0 or len(mom) == 0:
            return None

        signals = {}
        last_tech = tech.iloc[-1]
        for col in ["sig_ema_trend", "sig_rsi", "sig_macd", "sig_bb"]:
            signals[col] = float(last_tech[col])
        signals["trend_strength"] = float(last_tech["trend_strength"])
        signals["ADX"] = float(last_tech["ADX"])
        signals["ATR"] = float(last_tech["ATR"])
        signals["EMA_50"] = float(last_tech["EMA_50"])
        signals["close"] = float(last_tech["Close"])

        last_mom = mom.iloc[-1]
        for col in ["sig_momentum_5d", "sig_momentum_20d", "sig_mean_reversion", "sig_volume"]:
            signals[col] = float(last_mom[col])

        signals["sig_sentiment"] = self.sentiment_signal.compute(history_df, headlines)
        signals["sig_chronos"] = self.chronos_signal.compute(history_df)
        return signals


# Test the signal hub on NVDA history
signal_hub = SignalHub(CONFIG, use_sentiment=False, use_chronos=False)
test_history = data_engine.get_history("NVDA")
test_signals = signal_hub.compute(test_history)

print("Signal Hub Test (NVDA, latest bar):\n")
if test_signals:
    for key, val in sorted(test_signals.items()):
        if key.startswith("sig_"):
            bar = "#" * int(abs(val) * 20)
            direction = "+" if val > 0 else "-"
            print(f"  {key:<25} {val:>+6.3f}  {direction}{bar}")
    print(f"\n  ADX: {test_signals['ADX']:.1f} | ATR: ${test_signals['ATR']:.2f} | Close: ${test_signals['close']:.2f}")

Signal Hub Test (NVDA, latest bar):

  sig_bb                    +0.229  +####
  sig_chronos               -0.068  -#
  sig_ema_trend             +1.000  +####################
  sig_macd                  +0.049  +
  sig_mean_reversion        +0.806  +################
  sig_momentum_20d          -0.210  -####
  sig_momentum_5d           -0.163  -###
  sig_rsi                   +0.053  +#
  sig_sentiment             -0.070  -#
  sig_volume                +0.039  +

  ADX: 11.7 | ATR: $6.41 | Close: $182.81


---
## 4. Regime-Aware Composite Scoring

Adapted from Notebook 06 for live, single-bar operation. The regime detector
classifies the current market state, then applies adaptive weights to each
signal category.

| Regime | ADX | Trend Weight | Reversion Weight |
|--------|-----|-------------|-----------------|
| **Trend Up** | >25, price > EMA50 | 1.5x | 0.3x |
| **Trend Down** | >25, price < EMA50 | 1.5x | 0.3x |
| **Ranging** | <20 | 0.3x | 1.5x |
| **Volatile** | 20-25 | 0.5x | 0.5x |

In [5]:
# Signal category definitions (same as NB06)
SIGNAL_CATEGORIES = {
    "sig_ema_trend":       ("trend",     1.0),
    "sig_macd":            ("trend",     0.8),
    "sig_momentum_5d":     ("momentum",  0.7),
    "sig_momentum_20d":    ("momentum",  0.5),
    "sig_volume":          ("momentum",  0.6),
    "sig_rsi":             ("reversion", 0.8),
    "sig_bb":              ("reversion", 0.7),
    "sig_mean_reversion":  ("reversion", 0.5),
    "sig_sentiment":       ("sentiment", 1.0),
    "sig_chronos":         ("forecast",  1.0),
}

REGIME_WEIGHTS = {
    "trend_up":   {"trend": 1.5, "momentum": 1.2, "reversion": 0.3, "sentiment": 1.0, "forecast": 1.0},
    "trend_down": {"trend": 1.5, "momentum": 1.2, "reversion": 0.3, "sentiment": 1.0, "forecast": 1.0},
    "ranging":    {"trend": 0.3, "momentum": 0.5, "reversion": 1.5, "sentiment": 1.2, "forecast": 0.8},
    "volatile":   {"trend": 0.5, "momentum": 0.5, "reversion": 0.5, "sentiment": 0.8, "forecast": 0.5},
}


def classify_regime(signals: dict) -> str:
    """Classify market regime from signal dictionary."""
    adx = signals.get("ADX", 20)
    close = signals.get("close", 0)
    ema50 = signals.get("EMA_50", close)
    if adx > 25 and close > ema50:
        return "trend_up"
    elif adx > 25 and close <= ema50:
        return "trend_down"
    elif adx < 20:
        return "ranging"
    else:
        return "volatile"


def compute_composite_score_live(signals: dict, regime: str) -> Tuple[float, float]:
    """Compute regime-adaptive composite score from a signal dictionary.
    Returns (composite_score, signal_agreement) both in [-1, +1] range."""
    regime_w = REGIME_WEIGHTS.get(regime, REGIME_WEIGHTS["volatile"])
    weighted_sum = 0.0
    weight_sum = 0.0
    signs = []

    for signal_name, (category, base_weight) in SIGNAL_CATEGORIES.items():
        if signal_name not in signals:
            continue
        total_weight = base_weight * regime_w.get(category, 1.0)
        weighted_sum += signals[signal_name] * total_weight
        weight_sum += total_weight
        signs.append(np.sign(signals[signal_name]))

    composite = float(np.clip(weighted_sum / max(weight_sum, 1e-8), -1, 1))
    agreement = abs(sum(signs)) / max(len(signs), 1)
    return composite, agreement


# Test regime scoring
if test_signals:
    regime = classify_regime(test_signals)
    score, agreement = compute_composite_score_live(test_signals, regime)
    print(f"NVDA Regime Scoring Test:")
    print(f"  Regime:      {regime}")
    print(f"  Composite:   {score:+.3f}")
    print(f"  Agreement:   {agreement:.2f}")
    print(f"  Action:      ", end="")
    if score > CONFIG.entry_threshold and agreement > CONFIG.min_agreement:
        print(f"BUY (score {score:+.3f} > {CONFIG.entry_threshold})")
    elif score < CONFIG.exit_threshold:
        print(f"SELL (score {score:+.3f} < {CONFIG.exit_threshold})")
    else:
        print(f"HOLD (score {score:+.3f} between thresholds)")

NVDA Regime Scoring Test:
  Regime:      ranging
  Composite:   +0.153
  Agreement:   0.20
  Action:      HOLD (score +0.153 between thresholds)


---
## 5. Order Management System

The `PaperTradingEngine` manages positions, risk, and trade execution. It
maintains an internal ledger and optionally mirrors orders to Alpaca.

In [6]:
@dataclass
class PaperPosition:
    """An open paper trading position."""
    ticker: str
    direction: str
    entry_date: pd.Timestamp
    entry_price: float
    shares: int
    stop_loss: float
    take_profit: float
    entry_score: float
    entry_regime: str
    bars_held: int = 0
    alpaca_order_id: str = ""


@dataclass
class PaperTrade:
    """A completed trade record."""
    ticker: str
    direction: str
    entry_date: pd.Timestamp
    exit_date: pd.Timestamp
    entry_price: float
    exit_price: float
    shares: int
    pnl: float
    pnl_pct: float
    exit_reason: str
    entry_score: float
    exit_score: float
    regime: str
    bars_held: int


class PaperTradingEngine:
    """Paper trading engine with position tracking and risk management.
    Maintains an internal ledger. When Alpaca client is provided,
    mirrors orders to the paper trading API."""

    def __init__(self, config, alpaca_client=None):
        self.config = config
        self.alpaca = alpaca_client
        self.capital = config.initial_capital
        self.positions = {}
        self.trades = []
        self.equity_snapshots = []
        self.signal_log = []

    def snapshot_equity(self, bar_index, bars, timestamp):
        """Record current portfolio equity."""
        unrealized = 0
        for ticker, pos in self.positions.items():
            if ticker in bars:
                current_price = bars[ticker].close
                unrealized += (current_price - pos.entry_price) * pos.shares
        self.equity_snapshots.append({
            "bar_index": bar_index, "date": timestamp,
            "equity": self.capital + unrealized,
            "cash": self.capital, "n_positions": len(self.positions),
            "unrealized_pnl": unrealized,
        })

    def check_exits(self, bars, signals_by_ticker):
        """Check all positions for exit conditions."""
        to_close = []
        for ticker, pos in self.positions.items():
            if ticker not in bars:
                continue
            bar = bars[ticker]
            signals = signals_by_ticker.get(ticker, {})
            regime = classify_regime(signals) if signals else "volatile"
            score, _ = compute_composite_score_live(signals, regime) if signals else (0, 0)

            exit_price = None
            exit_reason = None

            if bar.low <= pos.stop_loss:
                exit_price = pos.stop_loss
                exit_reason = "stop_loss"
            elif bar.high >= pos.take_profit:
                exit_price = pos.take_profit
                exit_reason = "take_profit"
            elif score < self.config.exit_threshold:
                exit_price = bar.close * (1 - self.config.slippage_pct)
                exit_reason = "signal_exit"
            elif pos.bars_held >= self.config.max_hold_bars:
                exit_price = bar.close * (1 - self.config.slippage_pct)
                exit_reason = "max_hold"

            if exit_price:
                pnl = (exit_price - pos.entry_price) * pos.shares
                pnl_pct = (exit_price / pos.entry_price - 1) * 100
                self.capital += pos.entry_price * pos.shares + pnl

                self.trades.append(PaperTrade(
                    ticker=ticker, direction=pos.direction,
                    entry_date=pos.entry_date, exit_date=bar.timestamp,
                    entry_price=pos.entry_price, exit_price=exit_price,
                    shares=pos.shares, pnl=pnl, pnl_pct=pnl_pct,
                    exit_reason=exit_reason, entry_score=pos.entry_score,
                    exit_score=score, regime=pos.entry_regime,
                    bars_held=pos.bars_held,
                ))
                to_close.append(ticker)

                if self.alpaca:
                    try:
                        order = MarketOrderRequest(
                            symbol=ticker, qty=pos.shares,
                            side=OrderSide.SELL, time_in_force=TimeInForce.DAY)
                        self.alpaca.submit_order(order)
                    except Exception as e:
                        print(f"  Alpaca sell failed for {ticker}: {e}")

        for ticker in to_close:
            del self.positions[ticker]
        return to_close

    def check_entries(self, bars, signals_by_ticker):
        """Check for new entry opportunities."""
        if len(self.positions) >= self.config.max_positions:
            return []

        candidates = []
        for ticker, signals in signals_by_ticker.items():
            if ticker in self.positions or ticker == self.config.benchmark:
                continue
            if ticker not in bars:
                continue
            regime = classify_regime(signals)
            score, agreement = compute_composite_score_live(signals, regime)
            if score > self.config.entry_threshold and agreement > self.config.min_agreement:
                candidates.append((ticker, score, agreement, regime, signals))

        candidates.sort(key=lambda x: x[1], reverse=True)
        slots = self.config.max_positions - len(self.positions)
        entered = []

        for ticker, score, agreement, regime, signals in candidates[:slots]:
            bar = bars[ticker]
            entry_price = bar.open * (1 + self.config.slippage_pct)
            atr = signals.get("ATR", entry_price * 0.02)

            stop_loss = entry_price - self.config.atr_stop_mult * atr
            take_profit = entry_price + self.config.atr_target_mult * atr

            risk_amount = self.capital * (self.config.risk_per_trade_pct / 100)
            risk_per_share = entry_price - stop_loss
            if risk_per_share <= 0:
                continue

            shares = max(1, int(risk_amount / risk_per_share))
            max_alloc = self.capital * (self.config.max_per_stock_pct / 100)
            shares = min(shares, max(1, int(max_alloc / entry_price)))

            cost = shares * entry_price
            if cost > self.capital * 0.95:
                continue

            self.capital -= cost
            self.positions[ticker] = PaperPosition(
                ticker=ticker, direction="long",
                entry_date=bar.timestamp, entry_price=entry_price,
                shares=shares, stop_loss=stop_loss, take_profit=take_profit,
                entry_score=score, entry_regime=regime,
            )
            entered.append(ticker)

            if self.alpaca:
                try:
                    order = MarketOrderRequest(
                        symbol=ticker, qty=shares,
                        side=OrderSide.BUY, time_in_force=TimeInForce.DAY)
                    result = self.alpaca.submit_order(order)
                    self.positions[ticker].alpaca_order_id = str(result.id)
                except Exception as e:
                    print(f"  Alpaca buy failed for {ticker}: {e}")

        return entered

    def close_all(self, bars):
        """Close all remaining positions at current prices."""
        for ticker, pos in list(self.positions.items()):
            if ticker in bars:
                exit_price = bars[ticker].close
                pnl = (exit_price - pos.entry_price) * pos.shares
                pnl_pct = (exit_price / pos.entry_price - 1) * 100
                self.capital += pos.entry_price * pos.shares + pnl
                self.trades.append(PaperTrade(
                    ticker=ticker, direction=pos.direction,
                    entry_date=pos.entry_date, exit_date=bars[ticker].timestamp,
                    entry_price=pos.entry_price, exit_price=exit_price,
                    shares=pos.shares, pnl=pnl, pnl_pct=pnl_pct,
                    exit_reason="session_end", entry_score=pos.entry_score,
                    exit_score=0, regime=pos.entry_regime, bars_held=pos.bars_held,
                ))
        self.positions.clear()

    def get_metrics(self):
        """Compute portfolio performance metrics."""
        if not self.trades:
            return {"total_trades": 0}

        pnls = [t.pnl for t in self.trades]
        winners = [p for p in pnls if p > 0]
        losers = [p for p in pnls if p <= 0]

        equity_df = pd.DataFrame(self.equity_snapshots)
        if len(equity_df) < 2:
            return {"total_trades": len(self.trades), "total_pnl": sum(pnls)}

        equity_df = equity_df.set_index("date")
        equity = equity_df["equity"]
        returns = equity.pct_change().dropna()
        peak = equity.expanding().max()
        drawdown = (equity - peak) / peak

        sharpe = (returns.mean() / returns.std() * np.sqrt(252)) if returns.std() > 0 else 0
        gross_profit = sum(winners) if winners else 0
        gross_loss = abs(sum(losers)) if losers else 1

        return {
            "total_trades": len(self.trades),
            "winners": len(winners),
            "losers": len(losers),
            "win_rate": len(winners) / len(self.trades) * 100,
            "total_pnl": sum(pnls),
            "total_return_pct": (equity.iloc[-1] / self.config.initial_capital - 1) * 100,
            "avg_trade_pnl": np.mean(pnls),
            "avg_win": np.mean(winners) if winners else 0,
            "avg_loss": np.mean(losers) if losers else 0,
            "profit_factor": gross_profit / gross_loss if gross_loss > 0 else float("inf"),
            "sharpe_ratio": sharpe,
            "max_drawdown_pct": drawdown.min() * 100,
            "avg_bars_held": np.mean([t.bars_held for t in self.trades]),
            "tickers_traded": len(set(t.ticker for t in self.trades)),
            "exit_reasons": pd.Series([t.exit_reason for t in self.trades]).value_counts().to_dict(),
        }


print("PaperTradingEngine ready.")
print(f"  Max positions: {CONFIG.max_positions}")
print(f"  Risk per trade: {CONFIG.risk_per_trade_pct}%")

PaperTradingEngine ready.
  Max positions: 5
  Risk per trade: 2.0%


---
## 6. Pre-Market Scanner

Ranks the universe by signal strength before the session begins.
In live mode, this would incorporate overnight news and pre-market data.

In [7]:
class PreMarketScanner:
    """Scan the stock universe and rank by composite signal strength."""

    def __init__(self, signal_hub, data_engine, config):
        self.signal_hub = signal_hub
        self.data_engine = data_engine
        self.config = config

    def scan(self, bar_index=None):
        """Scan all stocks and return ranked results."""
        results = []
        for ticker in self.config.universe:
            history = self.data_engine.get_history(ticker, bar_index)
            if history is None or len(history) < self.config.warmup_bars:
                continue
            signals = self.signal_hub.compute(history)
            if signals is None:
                continue
            regime = classify_regime(signals)
            score, agreement = compute_composite_score_live(signals, regime)
            results.append({
                "ticker": ticker, "close": signals["close"],
                "composite_score": score, "agreement": agreement,
                "regime": regime, "ADX": signals["ADX"], "ATR": signals["ATR"],
                "sig_ema_trend": signals.get("sig_ema_trend", 0),
                "sig_rsi": signals.get("sig_rsi", 0),
                "sig_momentum_5d": signals.get("sig_momentum_5d", 0),
                "sig_sentiment": signals.get("sig_sentiment", 0),
                "sig_chronos": signals.get("sig_chronos", 0),
                "actionable": score > self.config.entry_threshold and agreement > self.config.min_agreement,
            })
        results.sort(key=lambda x: x["composite_score"], reverse=True)
        return results

    def print_brief(self, results):
        """Print a formatted morning brief."""
        print(f"\n{'='*75}")
        print(f"  PRE-MARKET SCANNER  |  {len(results)} stocks scanned")
        print(f"{'='*75}\n")
        print(f"  {'Ticker':<8} {'Price':>8} {'Score':>7} {'Agree':>7} {'Regime':<12} {'Action':<10}")
        print(f"  {'-'*62}")
        for r in results:
            action = "BUY" if r["actionable"] else "--"
            icon = ">>>" if r["actionable"] else "   "
            print(f"  {icon} {r['ticker']:<5} ${r['close']:>7.2f} {r['composite_score']:>+6.3f} "
                  f"{r['agreement']:>6.2f} {r['regime']:<12} {action:<10}")
        actionable = [r for r in results if r["actionable"]]
        print(f"\n  Actionable: {len(actionable)} stocks meet entry criteria")
        if actionable:
            print(f"  Top pick: {actionable[0]['ticker']} (score: {actionable[0]['composite_score']:+.3f})")


# Run pre-market scan
scanner = PreMarketScanner(signal_hub, data_engine, CONFIG)
last_bar = data_engine.replayer.max_bars - 1 if data_engine.replayer else None
scan_results = scanner.scan(bar_index=last_bar)
scanner.print_brief(scan_results)


  PRE-MARKET SCANNER  |  8 stocks scanned

  Ticker      Price   Score   Agree Regime       Action    
  --------------------------------------------------------------
      NVDA  $ 182.81 +0.153   0.20 ranging      --        
      META  $ 639.77 +0.111   0.30 ranging      --        
      AAPL  $ 255.78 +0.059   0.00 volatile     --        
      TSLA  $ 417.44 -0.091   0.00 volatile     --        
      AMD   $ 207.32 -0.210   0.60 volatile     --        
      GOOGL $ 305.72 -0.288   0.10 trend_down   --        
      MSFT  $ 401.32 -0.342   0.20 trend_down   --        
      AMZN  $ 198.79 -0.557   0.40 trend_down   --        

  Actionable: 0 stocks meet entry criteria


---
## 7. Live Trading Loop

The main event loop: for each bar, compute signals across the universe,
check exits, check entries, log everything.

```
For each bar:
  1. Compute signals for all tickers
  2. Check existing positions for exits (stop/TP/signal/max_hold)
  3. Check for new entry opportunities
  4. Snapshot equity
  5. Log signals to database
  6. Increment bar counters
```

In [8]:
def run_trading_session(config, data_engine, signal_hub, alpaca_client=None,
                       verbose=True, progress_every=20):
    """Run a complete paper trading session.
    In simulation mode, replays all historical bars.
    Returns PaperTradingEngine with all trades, equity, and metrics."""

    engine = PaperTradingEngine(config, alpaca_client)

    print(f"{'='*60}")
    mode_label = "(SIMULATION)" if config.simulation_mode else "(LIVE)"
    print(f"  TRADING SESSION {mode_label}")
    print(f"{'='*60}")
    print(f"  Capital: ${config.initial_capital:,.0f} | Max positions: {config.max_positions}")
    print(f"  Universe: {config.universe}\n")

    t0 = time.time()
    bar_count = 0
    last_bars = None

    for bar_index, bars in data_engine.iter_bars(warmup=config.warmup_bars):
        bar_count += 1
        last_bars = bars

        # 1. Compute signals for all tickers
        signals_by_ticker = {}
        for ticker in config.universe:
            history = data_engine.get_history(ticker, bar_index)
            if history is not None and len(history) >= config.warmup_bars:
                signals = signal_hub.compute(history)
                if signals:
                    regime = classify_regime(signals)
                    score, agreement = compute_composite_score_live(signals, regime)
                    signals["composite_score"] = score
                    signals["signal_agreement"] = agreement
                    signals["regime"] = regime
                    signals_by_ticker[ticker] = signals

        # 2. Increment bars_held
        for pos in engine.positions.values():
            pos.bars_held += 1

        # 3. Check exits
        engine.check_exits(bars, signals_by_ticker)

        # 4. Check entries
        engine.check_entries(bars, signals_by_ticker)

        # 5. Snapshot equity
        timestamp = next(iter(bars.values())).timestamp if bars else None
        if timestamp:
            engine.snapshot_equity(bar_index, bars, timestamp)

        # 6. Log signals
        for ticker, signals in signals_by_ticker.items():
            engine.signal_log.append({
                "bar_index": bar_index, "date": str(timestamp),
                "ticker": ticker,
                "composite_score": signals.get("composite_score", 0),
                "regime": signals.get("regime", ""),
                "agreement": signals.get("signal_agreement", 0),
            })

        # Progress reporting
        if verbose and bar_count % progress_every == 0:
            eq = engine.equity_snapshots[-1]["equity"] if engine.equity_snapshots else config.initial_capital
            n_pos = len(engine.positions)
            print(f"  Bar {bar_count:>4} | {str(timestamp)[:10]} | "
                  f"Equity: ${eq:>10,.0f} | Positions: {n_pos} | "
                  f"Trades: {len(engine.trades)}")

    # Close remaining positions
    if last_bars:
        engine.close_all(last_bars)

    elapsed = time.time() - t0
    m = engine.get_metrics()

    print(f"\n{'='*60}")
    print(f"  SESSION COMPLETE ({elapsed:.1f}s, {bar_count} bars)")
    print(f"{'='*60}")
    if m.get("total_trades", 0) > 0:
        print(f"  Total Trades:   {m['total_trades']}")
        print(f"  Win / Lose:     {m.get('winners', 0)} / {m.get('losers', 0)}")
        print(f"  Win Rate:       {m.get('win_rate', 0):.1f}%")
        print(f"  Total P&L:      ${m.get('total_pnl', 0):>+10,.2f}")
        print(f"  Total Return:   {m.get('total_return_pct', 0):>+10.2f}%")
        print(f"  Sharpe Ratio:   {m.get('sharpe_ratio', 0):>10.2f}")
        print(f"  Max Drawdown:   {m.get('max_drawdown_pct', 0):>10.2f}%")
        print(f"  Profit Factor:  {m.get('profit_factor', 0):>10.2f}")
        print(f"  Avg Hold (bars):{m.get('avg_bars_held', 0):>10.1f}")
        print(f"  Exit Reasons:   {m.get('exit_reasons', {})}")
    else:
        print("  No trades executed.")

    return engine


print("Trading session function ready.")
print("Run with: engine = run_trading_session(CONFIG, data_engine, signal_hub)")

Trading session function ready.
Run with: engine = run_trading_session(CONFIG, data_engine, signal_hub)


---
## 8. Interactive Dashboard (Plotly)

A 4-panel interactive dashboard showing:
1. **Equity curve** with drawdown overlay
2. **P&L by ticker** (bar chart)
3. **Signal heatmap** over time
4. **Trade log table**

In [9]:
def build_dashboard(engine, title="Paper Trading Dashboard"):
    """Build an interactive Plotly dashboard from trading results.
    Falls back to matplotlib if plotly is not available."""
    if not engine.trades:
        print("No trades to display.")
        return

    equity_df = pd.DataFrame(engine.equity_snapshots)
    trade_df = pd.DataFrame([{
        "ticker": t.ticker, "entry": str(t.entry_date)[:10],
        "exit": str(t.exit_date)[:10], "pnl": t.pnl,
        "pnl_pct": t.pnl_pct, "exit_reason": t.exit_reason,
        "bars_held": t.bars_held, "regime": t.regime,
    } for t in engine.trades])

    if not HAS_PLOTLY:
        print("Plotly not available -- using matplotlib fallback.")
        _dashboard_matplotlib(engine, equity_df, trade_df, title)
        return

    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=("Equity Curve", "P&L by Ticker",
                        "Signal Scores Over Time", "Trade Log"),
        specs=[[{"type": "scatter"}, {"type": "bar"}],
               [{"type": "heatmap"}, {"type": "table"}]],
        vertical_spacing=0.12, horizontal_spacing=0.08,
    )

    # 1. Equity curve
    if len(equity_df) > 0:
        fig.add_trace(go.Scatter(
            x=equity_df["date"], y=equity_df["equity"],
            mode="lines", name="Equity",
            line=dict(color="cyan", width=2),
        ), row=1, col=1)
        fig.add_hline(y=engine.config.initial_capital, line_dash="dash",
                      line_color="yellow", opacity=0.5, row=1, col=1)

    # 2. P&L by ticker
    by_ticker = trade_df.groupby("ticker")["pnl"].sum().sort_values()
    fig.add_trace(go.Bar(
        x=by_ticker.values, y=by_ticker.index, orientation="h",
        marker_color=["lime" if v > 0 else "red" for v in by_ticker.values],
        name="P&L",
    ), row=1, col=2)

    # 3. Signal heatmap
    signal_df = pd.DataFrame(engine.signal_log)
    if len(signal_df) > 0:
        pivot = signal_df.pivot_table(
            index="ticker", columns="date", values="composite_score", aggfunc="last")
        if pivot.shape[1] > 40:
            pivot = pivot.iloc[:, ::pivot.shape[1] // 40]
        fig.add_trace(go.Heatmap(
            z=pivot.values,
            x=[str(d)[:10] for d in pivot.columns],
            y=pivot.index.tolist(),
            colorscale="RdYlGn", zmid=0, zmin=-1, zmax=1,
            name="Composite Score",
        ), row=2, col=1)

    # 4. Trade log table
    if len(trade_df) > 0:
        display_df = trade_df.tail(20)
        fig.add_trace(go.Table(
            header=dict(
                values=["Ticker", "Entry", "Exit", "P&L", "P&L%", "Reason"],
                fill_color="rgb(30,30,30)", font=dict(color="white"),
            ),
            cells=dict(
                values=[
                    display_df["ticker"], display_df["entry"], display_df["exit"],
                    display_df["pnl"].apply(lambda x: f"${x:+,.0f}"),
                    display_df["pnl_pct"].apply(lambda x: f"{x:+.1f}%"),
                    display_df["exit_reason"],
                ],
                fill_color="rgb(20,20,20)", font=dict(color="white"),
            ),
        ), row=2, col=2)

    fig.update_layout(title=title, template="plotly_dark",
                      height=800, showlegend=False)
    fig.show()
    print("Interactive dashboard rendered.")


def _dashboard_matplotlib(engine, equity_df, trade_df, title):
    """Matplotlib fallback for the dashboard."""
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))

    if len(equity_df) > 0:
        axes[0, 0].plot(equity_df["date"], equity_df["equity"], color="cyan", linewidth=1.5)
        axes[0, 0].axhline(y=engine.config.initial_capital, color="yellow",
                           linestyle="--", alpha=0.3)
    axes[0, 0].set_title("Equity Curve")
    axes[0, 0].set_ylabel("Equity ($)")

    by_ticker = trade_df.groupby("ticker")["pnl"].sum().sort_values()
    colors = ["lime" if v > 0 else "red" for v in by_ticker.values]
    axes[0, 1].barh(by_ticker.index, by_ticker.values, color=colors, alpha=0.8)
    axes[0, 1].axvline(x=0, color="white", alpha=0.3)
    axes[0, 1].set_title("P&L by Ticker")

    axes[1, 0].hist([t.pnl for t in engine.trades], bins=25,
                    color="cyan", alpha=0.7, edgecolor="white", linewidth=0.5)
    axes[1, 0].axvline(x=0, color="yellow", linestyle="--", alpha=0.5)
    axes[1, 0].set_title("Trade P&L Distribution")
    axes[1, 0].set_xlabel("P&L ($)")

    reasons = trade_df["exit_reason"].value_counts()
    axes[1, 1].pie(reasons.values, labels=reasons.index, autopct="%1.0f%%",
                   colors=sns.color_palette("bright", len(reasons)))
    axes[1, 1].set_title("Exit Reasons")

    plt.suptitle(title, fontsize=14)
    plt.tight_layout()
    plt.show()


print("Dashboard builder ready.")
print(f"  Using: {'Plotly (interactive)' if HAS_PLOTLY else 'Matplotlib (static)'}")

Dashboard builder ready.
  Using: Plotly (interactive)


---
## 9. Trade Persistence (SQLite)

Store all trades, signals, and daily summaries in a SQLite database for
post-session analysis. Uses **peewee** ORM if available, falls back to
raw **sqlite3**.

In [10]:
class DBManager:
    """SQLite database manager for paper trade persistence."""

    def __init__(self, db_path="paper_trades.db"):
        self.db_path = db_path
        self.conn = sqlite3.connect(db_path)
        self._create_tables()
        print(f"Database ready: {db_path}")

    def _create_tables(self):
        cursor = self.conn.cursor()
        cursor.execute(
            "CREATE TABLE IF NOT EXISTS trades ("
            "id INTEGER PRIMARY KEY AUTOINCREMENT, "
            "ticker TEXT, direction TEXT, "
            "entry_date TEXT, exit_date TEXT, "
            "entry_price REAL, exit_price REAL, "
            "shares INTEGER, pnl REAL, pnl_pct REAL, "
            "exit_reason TEXT, entry_score REAL, exit_score REAL, "
            "regime TEXT, bars_held INTEGER, "
            "session_id TEXT, created_at TEXT DEFAULT CURRENT_TIMESTAMP)"
        )
        cursor.execute(
            "CREATE TABLE IF NOT EXISTS signals ("
            "id INTEGER PRIMARY KEY AUTOINCREMENT, "
            "bar_index INTEGER, date TEXT, ticker TEXT, "
            "composite_score REAL, regime TEXT, agreement REAL, "
            "session_id TEXT)"
        )
        cursor.execute(
            "CREATE TABLE IF NOT EXISTS daily_summary ("
            "id INTEGER PRIMARY KEY AUTOINCREMENT, "
            "date TEXT, equity REAL, cash REAL, "
            "n_positions INTEGER, unrealized_pnl REAL, "
            "session_id TEXT)"
        )
        self.conn.commit()

    def save_session(self, engine, session_id=None):
        """Save a complete trading session to the database."""
        if session_id is None:
            session_id = datetime.now().strftime("%Y%m%d_%H%M%S")

        cursor = self.conn.cursor()

        for t in engine.trades:
            cursor.execute(
                "INSERT INTO trades (ticker, direction, entry_date, exit_date, "
                "entry_price, exit_price, shares, pnl, pnl_pct, exit_reason, "
                "entry_score, exit_score, regime, bars_held, session_id) "
                "VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)",
                (t.ticker, t.direction, str(t.entry_date), str(t.exit_date),
                 t.entry_price, t.exit_price, t.shares, t.pnl, t.pnl_pct,
                 t.exit_reason, t.entry_score, t.exit_score, t.regime,
                 t.bars_held, session_id))

        for i, s in enumerate(engine.signal_log):
            if i % 5 == 0:
                cursor.execute(
                    "INSERT INTO signals (bar_index, date, ticker, composite_score, "
                    "regime, agreement, session_id) VALUES (?,?,?,?,?,?,?)",
                    (s["bar_index"], s["date"], s["ticker"],
                     s["composite_score"], s["regime"], s["agreement"], session_id))

        for snap in engine.equity_snapshots:
            cursor.execute(
                "INSERT INTO daily_summary (date, equity, cash, n_positions, "
                "unrealized_pnl, session_id) VALUES (?,?,?,?,?,?)",
                (str(snap["date"]), snap["equity"], snap["cash"],
                 snap["n_positions"], snap["unrealized_pnl"], session_id))

        self.conn.commit()
        n_trades = len(engine.trades)
        n_signals = len(engine.signal_log) // 5
        n_snaps = len(engine.equity_snapshots)
        print(f"Session saved: {n_trades} trades, ~{n_signals} signals, {n_snaps} equity snapshots")
        return session_id

    def load_trades(self, session_id=None):
        """Load trades from database as a DataFrame."""
        query = "SELECT * FROM trades"
        if session_id:
            query += f" WHERE session_id = ?"
            return pd.read_sql(query, self.conn, params=[session_id])
        return pd.read_sql(query, self.conn)

    def load_equity(self, session_id=None):
        """Load equity curve from database."""
        query = "SELECT * FROM daily_summary"
        if session_id:
            query += " WHERE session_id = ?"
            return pd.read_sql(query, self.conn, params=[session_id])
        return pd.read_sql(query, self.conn)

    def list_sessions(self):
        """List all saved sessions."""
        query = (
            "SELECT session_id, COUNT(*) as n_trades, "
            "MIN(entry_date) as first_trade, MAX(exit_date) as last_trade, "
            "ROUND(SUM(pnl), 2) as total_pnl "
            "FROM trades GROUP BY session_id ORDER BY session_id DESC"
        )
        return pd.read_sql(query, self.conn)

    def close(self):
        self.conn.close()


db = DBManager(CONFIG.db_path)
sessions = db.list_sessions()
if len(sessions) > 0:
    print(f"\nExisting sessions:")
    print(sessions.to_string(index=False))
else:
    print("No previous sessions found.")

Database ready: paper_trades.db

Existing sessions:
     session_id  n_trades         first_trade          last_trade  total_pnl
20260214_173351        25 2025-06-30 00:00:00 2026-02-13 00:00:00   -5871.22
20260214_173109        25 2025-06-30 00:00:00 2026-02-13 00:00:00   -5871.22
20260214_172417        25 2025-06-30 00:00:00 2026-02-13 00:00:00   -5871.22
20260214_172159        25 2025-06-30 00:00:00 2026-02-13 00:00:00   -5871.22
20260214_172002        25 2025-06-30 00:00:00 2026-02-13 00:00:00   -5871.22
20260214_171854        25 2025-06-30 00:00:00 2026-02-13 00:00:00   -5871.22
20260214_171725        25 2025-06-30 00:00:00 2026-02-13 00:00:00   -5871.22
20260214_171508        25 2025-06-30 00:00:00 2026-02-13 00:00:00   -5871.22
20260214_171307        25 2025-06-30 00:00:00 2026-02-13 00:00:00   -5871.22


---
## 10. QuantStats Performance Analysis

Generate a professional tearsheet comparing our paper trading performance
against the SPY benchmark. QuantStats provides:
- Sharpe, Sortino, Calmar ratios
- Monthly returns heatmap
- Drawdown analysis
- Rolling statistics

In [11]:
def generate_quantstats_report(engine, benchmark_ticker="SPY", data_engine=None):
    """Generate QuantStats performance analysis.
    Creates a tearsheet comparing strategy returns vs benchmark."""
    if not engine.equity_snapshots:
        print("No equity data to analyze.")
        return None

    equity_df = pd.DataFrame(engine.equity_snapshots).set_index("date")
    equity_df.index = pd.to_datetime(equity_df.index)
    strategy_returns = equity_df["equity"].pct_change().dropna()
    strategy_returns.name = "Strategy"

    benchmark_returns = None
    if data_engine:
        bench_data = data_engine.get_history(benchmark_ticker)
        if bench_data is not None:
            bench_returns = bench_data["Close"].pct_change().dropna()
            common_idx = strategy_returns.index.intersection(bench_returns.index)
            if len(common_idx) > 10:
                benchmark_returns = bench_returns.loc[common_idx]
                benchmark_returns.name = benchmark_ticker
                strategy_returns = strategy_returns.loc[common_idx]

    if not HAS_QUANTSTATS:
        print("QuantStats not available -- generating basic analysis.\n")
        _basic_performance_report(strategy_returns, benchmark_returns)
        return strategy_returns

    print("Generating QuantStats analysis...\n")
    qs.extend_pandas()

    print(f"{'='*55}")
    print(f"  QUANTSTATS PERFORMANCE REPORT")
    print(f"{'='*55}\n")

    print("Key Metrics:")
    print(f"  Sharpe Ratio:    {qs.stats.sharpe(strategy_returns):.2f}")
    print(f"  Sortino Ratio:   {qs.stats.sortino(strategy_returns):.2f}")
    print(f"  Max Drawdown:    {qs.stats.max_drawdown(strategy_returns)*100:.2f}%")
    print(f"  Calmar Ratio:    {qs.stats.calmar(strategy_returns):.2f}")
    print(f"  Win Rate:        {qs.stats.win_rate(strategy_returns)*100:.1f}%")
    print(f"  Volatility:      {qs.stats.volatility(strategy_returns)*100:.2f}%")

    if benchmark_returns is not None:
        print(f"\n  vs {benchmark_ticker}:")
        print(f"  Benchmark Sharpe: {qs.stats.sharpe(benchmark_returns):.2f}")
        print(f"  Information Ratio: {qs.stats.information_ratio(strategy_returns, benchmark_returns):.2f}")

    # Inline performance plots
    try:
        fig, axes = plt.subplots(2, 2, figsize=(16, 10))

        cum_ret = (1 + strategy_returns).cumprod()
        axes[0, 0].plot(cum_ret.index, cum_ret.values, color="cyan", linewidth=1.5, label="Strategy")
        if benchmark_returns is not None:
            bench_cum = (1 + benchmark_returns).cumprod()
            axes[0, 0].plot(bench_cum.index, bench_cum.values, color="yellow",
                           linewidth=1, alpha=0.7, label=benchmark_ticker)
        axes[0, 0].legend(fontsize=8)
        axes[0, 0].set_title("Cumulative Returns")
        axes[0, 0].set_ylabel("Growth of $1")

        monthly = strategy_returns.resample("ME").apply(lambda x: (1+x).prod()-1) * 100
        colors = ["lime" if r > 0 else "red" for r in monthly.values]
        axes[0, 1].bar(range(len(monthly)), monthly.values, color=colors, alpha=0.7)
        axes[0, 1].axhline(y=0, color="white", alpha=0.3)
        axes[0, 1].set_title("Monthly Returns (%)")

        cum = (1 + strategy_returns).cumprod()
        peak = cum.expanding().max()
        dd = (cum - peak) / peak * 100
        axes[1, 0].fill_between(dd.index, dd.values, 0, color="red", alpha=0.4)
        axes[1, 0].set_title("Drawdown (%)")

        rolling_sharpe = strategy_returns.rolling(30).apply(
            lambda x: x.mean() / x.std() * np.sqrt(252) if x.std() > 0 else 0)
        axes[1, 1].plot(rolling_sharpe.index, rolling_sharpe.values, color="orange", linewidth=1)
        axes[1, 1].axhline(y=0, color="white", alpha=0.3)
        axes[1, 1].axhline(y=1, color="lime", linestyle="--", alpha=0.3, label="Sharpe=1")
        axes[1, 1].set_title("Rolling 30-Day Sharpe")
        axes[1, 1].legend(fontsize=8)

        plt.suptitle("QuantStats Performance Snapshot", fontsize=14)
        plt.tight_layout()
        plt.show()
    except Exception as e:
        print(f"Plot generation error: {e}")

    return strategy_returns


def _basic_performance_report(strategy_returns, benchmark_returns=None):
    """Fallback performance report without quantstats."""
    total_ret = (1 + strategy_returns).prod() - 1
    sharpe = strategy_returns.mean() / strategy_returns.std() * np.sqrt(252) if strategy_returns.std() > 0 else 0
    cum = (1 + strategy_returns).cumprod()
    peak = cum.expanding().max()
    max_dd = ((cum - peak) / peak).min()

    print(f"  Total Return:  {total_ret*100:+.2f}%")
    print(f"  Sharpe Ratio:  {sharpe:.2f}")
    print(f"  Max Drawdown:  {max_dd*100:.2f}%")
    print(f"  Volatility:    {strategy_returns.std()*np.sqrt(252)*100:.2f}%")

    if benchmark_returns is not None:
        bench_ret = (1 + benchmark_returns).prod() - 1
        print(f"  Benchmark Ret: {bench_ret*100:+.2f}%")
        print(f"  Excess Return: {(total_ret - bench_ret)*100:+.2f}%")


print("QuantStats analysis ready.")
print(f"  Using: {'quantstats' if HAS_QUANTSTATS else 'basic fallback'}")

QuantStats analysis ready.
  Using: quantstats


---
## 11. Statistical Validation

Rigorous statistical tests to determine if the strategy's performance
is statistically significant or could be due to chance.

| Test | Purpose |
|------|---------|
| **One-sample t-test** | Are mean returns significantly different from zero? |
| **Bootstrap Sharpe CI** | Confidence interval for the Sharpe ratio |
| **Mann-Whitney U** | Do returns differ between regimes? |
| **QQ Plot** | Are returns normally distributed? |

In [12]:
def statistical_validation(engine):
    """Run statistical validation tests on trading results."""
    if not engine.trades:
        print("No trades to validate.")
        return

    trade_returns = np.array([t.pnl_pct / 100 for t in engine.trades])
    n = len(trade_returns)

    print(f"{'='*55}")
    print(f"  STATISTICAL VALIDATION  ({n} trades)")
    print(f"{'='*55}\n")

    if not HAS_SCIPY:
        print("scipy not available -- skipping statistical tests.")
        print("Install with: pip install scipy")
        return

    # 1. One-sample t-test
    t_stat, p_value = scipy_stats.ttest_1samp(trade_returns, 0)
    sig = "***" if p_value < 0.01 else "**" if p_value < 0.05 else "*" if p_value < 0.10 else ""
    print(f"1. One-Sample t-test (H0: mean return = 0)")
    print(f"   t-statistic: {t_stat:.3f}")
    print(f"   p-value:     {p_value:.4f} {sig}")
    print(f"   Mean return:  {np.mean(trade_returns)*100:+.3f}%")
    if p_value < 0.05:
        print(f"   -> Returns are statistically significant at 5% level.")
    else:
        print(f"   -> Cannot reject null hypothesis. Returns may be due to chance.")

    # 2. Bootstrap Sharpe CI
    print(f"\n2. Bootstrap Sharpe Ratio (1000 resamples)")
    n_bootstrap = 1000
    bootstrap_sharpes = []
    for _ in range(n_bootstrap):
        sample = np.random.choice(trade_returns, size=n, replace=True)
        if sample.std() > 0:
            sharpe = sample.mean() / sample.std() * np.sqrt(252)
        else:
            sharpe = 0
        bootstrap_sharpes.append(sharpe)

    ci_low = np.percentile(bootstrap_sharpes, 2.5)
    ci_high = np.percentile(bootstrap_sharpes, 97.5)
    median_sharpe = np.median(bootstrap_sharpes)
    print(f"   Median Sharpe: {median_sharpe:.2f}")
    print(f"   95% CI:        [{ci_low:.2f}, {ci_high:.2f}]")
    if ci_low > 0:
        print(f"   -> Sharpe is significantly positive (CI excludes 0).")
    else:
        print(f"   -> Sharpe CI includes 0. Positive performance not guaranteed.")

    # 3. Mann-Whitney: regime comparison
    print(f"\n3. Mann-Whitney U Test (Regime Comparison)")
    trending_returns = [t.pnl_pct / 100 for t in engine.trades
                       if t.regime in ("trend_up", "trend_down")]
    other_returns = [t.pnl_pct / 100 for t in engine.trades
                    if t.regime not in ("trend_up", "trend_down")]

    if len(trending_returns) >= 5 and len(other_returns) >= 5:
        u_stat, p_mw = scipy_stats.mannwhitneyu(
            trending_returns, other_returns, alternative="two-sided")
        print(f"   Trending trades: {len(trending_returns)} (mean: {np.mean(trending_returns)*100:+.2f}%)")
        print(f"   Other trades:    {len(other_returns)} (mean: {np.mean(other_returns)*100:+.2f}%)")
        print(f"   U-statistic: {u_stat:.1f}, p-value: {p_mw:.4f}")
        if p_mw < 0.05:
            print(f"   -> Significant difference between regime performance.")
        else:
            print(f"   -> No significant difference between regimes.")
    else:
        print(f"   Insufficient trades per regime for comparison.")

    # 4. Normality test
    print(f"\n4. Shapiro-Wilk Normality Test")
    test_sample = trade_returns[:min(n, 500)]
    w_stat, p_norm = scipy_stats.shapiro(test_sample)
    print(f"   W-statistic: {w_stat:.4f}, p-value: {p_norm:.4f}")
    if p_norm < 0.05:
        print(f"   -> Returns are NOT normally distributed (common for trading).")
    else:
        print(f"   -> Returns appear normally distributed.")

    # Visualizations
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))

    scipy_stats.probplot(trade_returns, dist="norm", plot=axes[0, 0])
    axes[0, 0].set_title("QQ Plot (Returns vs Normal)")
    axes[0, 0].get_lines()[0].set_color("cyan")
    axes[0, 0].get_lines()[1].set_color("yellow")

    axes[0, 1].hist(trade_returns * 100, bins=30, density=True,
                    color="cyan", alpha=0.7, edgecolor="white", linewidth=0.5)
    x = np.linspace(trade_returns.min() * 100, trade_returns.max() * 100, 100)
    axes[0, 1].plot(x, scipy_stats.norm.pdf(x, np.mean(trade_returns)*100,
                    np.std(trade_returns)*100),
                    color="yellow", linewidth=2, label="Normal fit")
    axes[0, 1].set_title("Return Distribution vs Normal")
    axes[0, 1].set_xlabel("Return (%)")
    axes[0, 1].legend(fontsize=8)

    axes[1, 0].hist(bootstrap_sharpes, bins=40, color="orange", alpha=0.7,
                    edgecolor="white", linewidth=0.5)
    axes[1, 0].axvline(x=ci_low, color="red", linestyle="--", label=f"95% CI: [{ci_low:.2f}, {ci_high:.2f}]")
    axes[1, 0].axvline(x=ci_high, color="red", linestyle="--")
    axes[1, 0].axvline(x=0, color="yellow", linestyle="--", alpha=0.5, label="Zero")
    axes[1, 0].set_title("Bootstrap Sharpe Distribution")
    axes[1, 0].set_xlabel("Sharpe Ratio")
    axes[1, 0].legend(fontsize=8)

    regime_groups = defaultdict(list)
    for t in engine.trades:
        regime_groups[t.regime].append(t.pnl_pct)
    if regime_groups:
        labels = list(regime_groups.keys())
        data = [regime_groups[k] for k in labels]
        bp = axes[1, 1].boxplot(data, labels=labels, patch_artist=True)
        colors_map = {"trend_up": "lime", "trend_down": "red",
                     "ranging": "yellow", "volatile": "gray"}
        for patch, label in zip(bp["boxes"], labels):
            patch.set_facecolor(colors_map.get(label, "cyan"))
            patch.set_alpha(0.6)
        axes[1, 1].axhline(y=0, color="white", alpha=0.3)
        axes[1, 1].set_title("Returns by Regime")
        axes[1, 1].set_ylabel("Return (%)")

    plt.suptitle("Statistical Validation", fontsize=14)
    plt.tight_layout()
    plt.show()

    return {"t_test_p": p_value, "sharpe_ci": (ci_low, ci_high), "normality_p": p_norm}


print("Statistical validation ready.")

Statistical validation ready.


---
## 12. Risk Management Dashboard

Comprehensive risk analysis with:
- Drawdown with alert zones (yellow >5%, red >10%)
- P&L concentration by ticker
- Exit reason breakdown
- Rolling 30-bar Sharpe ratio

In [13]:
def risk_dashboard(engine):
    """Generate a risk management dashboard from trading results."""
    if not engine.trades or not engine.equity_snapshots:
        print("No data for risk dashboard.")
        return

    equity_df = pd.DataFrame(engine.equity_snapshots).set_index("date")
    equity_df.index = pd.to_datetime(equity_df.index)

    fig = plt.figure(figsize=(16, 14))
    gs = gridspec.GridSpec(3, 2, hspace=0.35, wspace=0.3)

    # 1. Drawdown with alert zones
    ax1 = fig.add_subplot(gs[0, :])
    equity = equity_df["equity"]
    peak = equity.expanding().max()
    dd = (equity - peak) / peak * 100
    ax1.fill_between(dd.index, dd.values, 0, color="red", alpha=0.3)
    ax1.axhline(y=-5, color="yellow", linestyle="--", alpha=0.7, label="Warning (-5%)")
    ax1.axhline(y=-10, color="red", linestyle="--", alpha=0.7, label="Danger (-10%)")
    ax1.axhline(y=-15, color="magenta", linestyle="--", alpha=0.7, label="Critical (-15%)")
    ax1.set_ylabel("Drawdown (%)")
    ax1.set_title("Drawdown with Alert Zones")
    ax1.legend(fontsize=8)
    ax1.fill_between(dd.index, -5, 0, alpha=0.05, color="green")
    ax1.fill_between(dd.index, -10, -5, alpha=0.05, color="yellow")
    ax1.fill_between(dd.index, dd.values.min() - 2, -10, alpha=0.05, color="red")

    # 2. P&L concentration
    ax2 = fig.add_subplot(gs[1, 0])
    trade_pnls = pd.DataFrame([{"ticker": t.ticker, "pnl": abs(t.pnl)} for t in engine.trades])
    if len(trade_pnls) > 0:
        concentration = trade_pnls.groupby("ticker")["pnl"].sum().sort_values(ascending=False)
        cum_pct = concentration.cumsum() / concentration.sum() * 100
        ax2.bar(range(len(concentration)), concentration.values, color="cyan", alpha=0.8)
        ax2_twin = ax2.twinx()
        ax2_twin.plot(range(len(cum_pct)), cum_pct.values, color="yellow",
                     marker="o", linewidth=2)
        ax2_twin.axhline(y=80, color="red", linestyle="--", alpha=0.5)
        ax2_twin.set_ylabel("Cumulative %", color="yellow")
        ax2.set_xticks(range(len(concentration)))
        ax2.set_xticklabels(concentration.index, rotation=45, ha="right", fontsize=8)
    ax2.set_title("P&L Concentration (Absolute)")
    ax2.set_ylabel("Absolute P&L ($)")

    # 3. Exit reason pie
    ax3 = fig.add_subplot(gs[1, 1])
    reasons = pd.Series([t.exit_reason for t in engine.trades]).value_counts()
    colors_list = ["red", "lime", "cyan", "yellow", "magenta", "orange"]
    ax3.pie(reasons.values, labels=reasons.index, autopct="%1.0f%%",
            colors=colors_list[:len(reasons)], textprops={"fontsize": 9})
    ax3.set_title("Exit Reasons")

    # 4. Rolling 30-bar Sharpe
    ax4 = fig.add_subplot(gs[2, 0])
    returns = equity.pct_change().dropna()
    if len(returns) > 30:
        rolling_sharpe = returns.rolling(30).apply(
            lambda x: x.mean() / x.std() * np.sqrt(252) if x.std() > 0 else 0)
        ax4.plot(rolling_sharpe.index, rolling_sharpe.values, color="orange", linewidth=1.5)
        ax4.axhline(y=0, color="white", alpha=0.3)
        ax4.axhline(y=1, color="lime", linestyle="--", alpha=0.3, label="Sharpe=1")
        ax4.axhline(y=-1, color="red", linestyle="--", alpha=0.3, label="Sharpe=-1")
        ax4.fill_between(rolling_sharpe.index, rolling_sharpe.values, 0,
                        where=rolling_sharpe.values > 0, alpha=0.1, color="lime")
        ax4.fill_between(rolling_sharpe.index, rolling_sharpe.values, 0,
                        where=rolling_sharpe.values < 0, alpha=0.1, color="red")
    ax4.set_title("Rolling 30-Day Sharpe Ratio")
    ax4.set_ylabel("Sharpe")
    ax4.legend(fontsize=8)

    # 5. Win rate by holding period
    ax5 = fig.add_subplot(gs[2, 1])
    hold_groups = defaultdict(list)
    for t in engine.trades:
        if t.bars_held <= 3:
            bucket = "1-3 bars"
        elif t.bars_held <= 7:
            bucket = "4-7 bars"
        elif t.bars_held <= 14:
            bucket = "8-14 bars"
        else:
            bucket = "15+ bars"
        hold_groups[bucket].append(1 if t.pnl > 0 else 0)

    if hold_groups:
        buckets = ["1-3 bars", "4-7 bars", "8-14 bars", "15+ bars"]
        existing_buckets = [b for b in buckets if b in hold_groups]
        win_rates = [np.mean(hold_groups[b]) * 100 for b in existing_buckets]
        counts = [len(hold_groups[b]) for b in existing_buckets]
        bars_plot = ax5.bar(range(len(existing_buckets)), win_rates,
                          color="cyan", alpha=0.8)
        ax5.set_xticks(range(len(existing_buckets)))
        ax5.set_xticklabels(existing_buckets, fontsize=9)
        ax5.axhline(y=50, color="yellow", linestyle="--", alpha=0.5)
        ax5.set_ylabel("Win Rate (%)")
        ax5.set_title("Win Rate by Holding Period")
        for bar, count in zip(bars_plot, counts):
            ax5.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
                    f"n={count}", ha="center", fontsize=8, color="white")

    plt.suptitle("Risk Management Dashboard", fontsize=14, y=1.0)
    plt.tight_layout()
    plt.show()


print("Risk dashboard ready.")

Risk dashboard ready.


---
## 13. Full System Demo

End-to-end execution: data load -> pre-market scan -> trading session ->
database persistence -> QuantStats analysis -> statistical validation ->
risk dashboard -> interactive dashboard.

In [14]:
# ===================================================================
#  FULL SYSTEM DEMO
# ===================================================================

# 1. Pre-market scan
print("STEP 1: Pre-Market Scan\n")
scan_results = scanner.scan(bar_index=CONFIG.warmup_bars)
scanner.print_brief(scan_results)

# 2. Run trading session
print("\nSTEP 2: Trading Session\n")
engine = run_trading_session(
    CONFIG, data_engine, signal_hub,
    alpaca_client=alpaca_client,
    verbose=True, progress_every=30,
)

# 3. Save to database
print("\nSTEP 3: Persist to Database\n")
session_id = db.save_session(engine)

print("\nAll sessions:")
print(db.list_sessions().to_string(index=False))

STEP 1: Pre-Market Scan


  PRE-MARKET SCANNER  |  8 stocks scanned

  Ticker      Price   Score   Agree Regime       Action    
  --------------------------------------------------------------
      NVDA  $ 182.81 +0.153   0.20 ranging      --        
      META  $ 639.77 +0.111   0.30 ranging      --        
      AAPL  $ 255.78 +0.059   0.00 volatile     --        
      TSLA  $ 417.44 -0.091   0.00 volatile     --        
      AMD   $ 207.32 -0.210   0.60 volatile     --        
      GOOGL $ 305.72 -0.288   0.10 trend_down   --        
      MSFT  $ 401.32 -0.342   0.20 trend_down   --        
      AMZN  $ 198.79 -0.557   0.40 trend_down   --        

  Actionable: 0 stocks meet entry criteria

STEP 2: Trading Session

  TRADING SESSION (LIVE)
  Capital: $100,000 | Max positions: 5
  Universe: ['NVDA', 'AAPL', 'TSLA', 'AMD', 'MSFT', 'META', 'AMZN', 'GOOGL']


  SESSION COMPLETE (0.0s, 0 bars)
  No trades executed.

STEP 3: Persist to Database

Session saved: 0 trades, ~0 signals

In [15]:
# 4. QuantStats analysis
print("STEP 4: QuantStats Performance Analysis\n")
strategy_returns = generate_quantstats_report(engine, "SPY", data_engine)

# 5. Statistical validation
print("\nSTEP 5: Statistical Validation\n")
stats_results = statistical_validation(engine)

# 6. Risk management dashboard
print("\nSTEP 6: Risk Management Dashboard\n")
risk_dashboard(engine)

# 7. Interactive dashboard
print("\nSTEP 7: Interactive Dashboard\n")
build_dashboard(engine, title=f"Paper Trading Session: {session_id}")

# Final summary
m = engine.get_metrics()
print(f"\n{'='*60}")
print(f"  DEMO COMPLETE")
print(f"{'='*60}")
print(f"  Session: {session_id}")
print(f"  Trades:  {m.get('total_trades', 0)}")
print(f"  Return:  {m.get('total_return_pct', 0):+.2f}%")
print(f"  Sharpe:  {m.get('sharpe_ratio', 0):.2f}")
if stats_results:
    ci = stats_results.get("sharpe_ci", (0, 0))
    print(f"  Sharpe 95% CI: [{ci[0]:.2f}, {ci[1]:.2f}]")
print(f"  Database: {CONFIG.db_path}")
print(f"\n  All results persisted. Reload with:")
print(f"    db = DBManager('{CONFIG.db_path}')")
print(f"    trades = db.load_trades('{session_id}')")

STEP 4: QuantStats Performance Analysis

No equity data to analyze.

STEP 5: Statistical Validation

No trades to validate.

STEP 6: Risk Management Dashboard

No data for risk dashboard.

STEP 7: Interactive Dashboard

No trades to display.

  DEMO COMPLETE
  Session: 20260214_174011
  Trades:  0
  Return:  +0.00%
  Sharpe:  0.00
  Database: paper_trades.db

  All results persisted. Reload with:
    db = DBManager('paper_trades.db')
    trades = db.load_trades('20260214_174011')


---
## 14. Next Steps: Going Live

### From Paper to Real Money

This notebook gives you a **complete paper trading system**. Before going live,
work through this checklist carefully.

---

### Pre-Live Checklist

| Category | Item | Notes |
|----------|------|-------|
| **Psychology** | Paper trade for 30+ days | Track emotions, not just P&L |
| **Psychology** | Define max daily loss | Stop trading after hitting limit |
| **Psychology** | Accept that losses are normal | Even 50% win rate can be profitable |
| **Slippage** | Test with higher slippage (0.1-0.2%) | Real fills are worse than backtests |
| **PDT Rule** | Maintain $25K+ account | Pattern Day Trader rule applies to <$25K accounts |
| **Capital** | Never risk more than you can afford to lose | This is research, not financial advice |
| **Infrastructure** | Reliable internet + power backup | Missed exit signals can be costly |
| **Data Quality** | Verify data source reliability | Bad data = bad signals |
| **Taxes** | Understand wash sale rules | Short-term gains are taxed as income |
| **Taxes** | Track cost basis carefully | Your broker should handle this |

---

### Alpaca Go-Live Steps

```python
# 1. Create Alpaca account at https://alpaca.markets
# 2. Get API keys from the dashboard
# 3. Set environment variables:
#    export ALPACA_KEY="your-key"
#    export ALPACA_SECRET="your-secret"

# 4. Switch to live paper trading:
CONFIG.simulation_mode = False

# 5. Run the trading session:
engine = run_trading_session(CONFIG, data_engine, signal_hub)
```

---

### Enhancement Ideas

| Enhancement | Difficulty | Impact |
|-------------|-----------|--------|
| Real-time sentiment from news APIs | Medium | High |
| Live Chronos forecasts (not proxy) | Low | Medium |
| Intraday (5-min) bar support | High | High |
| Multi-timeframe confirmation | Medium | Medium |
| Sector rotation overlay | Medium | Medium |
| Options integration | High | High |
| Telegram/Discord alerts | Low | Medium |
| Web dashboard (Dash/Streamlit) | Medium | High |

---

### Series Complete!

You have built a comprehensive day trading research environment:

| Notebook | What You Built |
|----------|---------------|
| **01** | Market data, candlesticks, indicators, LSTM intro |
| **02** | Multi-model NLP sentiment pipeline |
| **03** | Custom backtesting engine with 5 strategies |
| **04** | Chronos zero-shot time series forecasting |
| **05** | Local LLM financial analysis (Phi-3, Mistral) |
| **06** | Multi-signal composite scoring, regime detection, Monte Carlo |
| **07** | Paper trading system tying it all together |

**Remember**: This is an educational research project. Past performance in
backtesting does not guarantee future results. Always practice responsible
risk management.

In [16]:
# Clean up
db.close()

if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print("GPU memory freed.")

print("\nNotebook 07 complete. Your paper trading system is ready.")
print("Run the full demo cell above to execute a complete session.")
print("\nTo go live with Alpaca paper trading:")
print("  1. pip install alpaca-py")
print("  2. export ALPACA_KEY='...' ALPACA_SECRET='...'")
print("  3. Set CONFIG.simulation_mode = False")
print("  4. Re-run the trading session cell")

GPU memory freed.

Notebook 07 complete. Your paper trading system is ready.
Run the full demo cell above to execute a complete session.

To go live with Alpaca paper trading:
  1. pip install alpaca-py
  2. export ALPACA_KEY='...' ALPACA_SECRET='...'
  3. Set CONFIG.simulation_mode = False
  4. Re-run the trading session cell


In [17]:
!nvidia-smi

Sat Feb 14 17:40:11 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.126.09             Driver Version: 580.126.09     CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4090        Off |   00000000:04:00.0 Off |                  Off |
|  0%   33C    P8              7W /  450W |      20MiB /  24564MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----